In [10]:
import numpy as np
import multiprocessing as mp

In [11]:
###########################################
############ Questões 1, 2 e 3 ############
###########################################

"""
Gera uma reta aleatória
"""
def generate_line():
    
    # Dimensões do poblema
    d = 2
    
    # Gera 2 pontos aleatórios entre 0 e 1 e os converte para
    # o intervalo entre -1 e 1
    p1 = np.random.rand(d) * 2 - 1
    p2 = np.random.rand(d) * 2 - 1
    
    # Calcula a reta na forma `y=ax + b` entre os pontos
    a = (p2[1] - p1[1]) / (p2[0] - p1[0])
    b = p1[1] - a*p1[0]

    return np.array([b, a, -1])


"""
Gera os dados baseados em uma reta
"""
def generate_data(n):
    
    # Gera 'n' pontos (x, y) aleatoriamente entre 0 e 1
    # e os converte para o intervalo -1 e 1
    points = (np.random.rand(n, 2) * 2) - 1
    
    return points

"""
Calcula os valores de saída da função ideal
"""
def calc_y(line, data):
    
    y = line[0] + np.dot(data, line[1:])
    
    # Transforma os valores em -1 ou 1
    # Valores em cima da reta são considerados -1
    y = np.where(y > 0, 1, -1)
    
    return y

def calc_log_y(x, w):
    
    m_x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)
    
    s = np.exp(np.dot(m_x, w))
    
    return s / (1 + s)

In [12]:
"""
Calcula a regressão linear
"""
def logistic_regression(x, y, l_rate=0.01, term_crit=0.01, max_epochs=5000):
    
    it = 0
    diff = float('inf')
    prev_w = np.zeros(3)
    w = prev_w
    
    m_x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)
    
    while diff > term_crit and it < max_epochs:
        
        indices = np.arange(y.shape[0])
        np.random.shuffle(indices)
    
        prev_w = w
        
        for n in indices:
            
            yn = y[n]
            xn = m_x[n]
            
            grad = -xn * (yn / (1 + np.exp( yn * np.dot(w, xn) )))
            
            w = w - l_rate * grad
            
        temp = w - prev_w

        it += 1
        
        diff = np.sqrt(temp.dot(temp)) 
        
    return [w, it]

In [20]:
def cross_entropy(y, hx):
    
    d = np.stack((y, hx), axis=-1)
    
    d = np.apply_along_axis(lambda x: -np.log(x[1]) if x[0] == 1 else -np.log(1 - x[1]), 1, d)
    
    return np.mean( d )

"""
Calcula o E_out para uma função linear
"""
def calc_log_e_out(line, w):
    
    # Gera mil pontos para serem avaliados
    ev_data = generate_data(1000)
    
    # Saída ideal
    y = calc_y(line, ev_data)
    
    hx = calc_log_y(ev_data, w)
    
    e_out = cross_entropy(y, hx)

    return np.sum(e_out)

In [14]:
###########################################
############### Experimentos ##############
###########################################

def experiment_1(N):
    
    np.random.seed()
    
    line = generate_line()

    x = generate_data(N)
    y = calc_y(line, x)
    
    w_log = logistic_regression(x, y)[0]
    
    e_out = calc_log_e_out(line, w_log)
    
    return e_out


def experiment_2(N):
    
    np.random.seed()
    
    line = generate_line()

    x = generate_data(N)
    y = calc_y(line, x)
    
    w_log = logistic_regression(x, y)[1]
    
    return w_log

In [15]:
"""
Executa um certo número de experimentos paralelamente
Caso o número de processos não seja espeficidado,
o multiprocessing utiliza o valor padrão,
que costuma ser o número de processadores
"""
def run_experiment(N, num_exp, exp_id, processes=None):
    pool = mp.Pool(processes)
    
    function = None
    
    if exp_id == 1:
        function = experiment_1
    elif exp_id == 2:
        function = experiment_2
    else:
        print('Invalid experiment!')
        return None
    
    # Executa os experimentos 'num_exp' vezes, passando como
    # parâmetro para cada um, o número de dados N a serem gerados
    results = np.array(pool.map(function, [N] * num_exp))
    
    pool.close()
    
    # Calcula a média dos resultados por coluna
    return np.mean(results, axis=0)

In [21]:
run_experiment(100, 100, 1)

0.10300236208103236

In [19]:
run_experiment(100, 100, 2)

341.99000000000001